# Word2Vec Tutorial
This tutorial follows a [blog post](http://rare-technologies.com/word2vec-tutorial/) written by the creator of gensim.

## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence a list of words (utf8 strings):

In [1]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2016-11-14 14:56:26,987 : INFO : collecting all words and their counts
2016-11-14 14:56:26,991 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-14 14:56:26,993 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2016-11-14 14:56:26,995 : INFO : Loading a fresh vocabulary
2016-11-14 14:56:26,996 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2016-11-14 14:56:26,997 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2016-11-14 14:56:26,999 : INFO : deleting the raw counts dictionary of 3 items
2016-11-14 14:56:27,002 : INFO : sample=0.001 downsamples 3 most-common words
2016-11-14 14:56:27,004 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2016-11-14 14:56:27,006 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2016-11-14 14:56:27,007 : INFO : resetting layer weights
2016-11-14 14:56:27,010 : INFO : training model with 3 workers o

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [3]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [5]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


In [6]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)
print(model)
print(model.vocab)

2016-11-14 14:56:45,918 : INFO : collecting all words and their counts
2016-11-14 14:56:45,920 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-14 14:56:45,921 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2016-11-14 14:56:45,923 : INFO : Loading a fresh vocabulary
2016-11-14 14:56:45,924 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2016-11-14 14:56:45,926 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2016-11-14 14:56:45,927 : INFO : deleting the raw counts dictionary of 3 items
2016-11-14 14:56:45,929 : INFO : sample=0.001 downsamples 3 most-common words
2016-11-14 14:56:45,929 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2016-11-14 14:56:45,930 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2016-11-14 14:56:45,931 : INFO : resetting layer weights
2016-11-14 14:56:45,932 : INFO : training model with 3 workers o

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.word2vec.Vocab object at 0x7f9b5eb8cd50>, 'first': <gensim.models.word2vec.Vocab object at 0x7f9b2dfb0d50>, 'sentence': <gensim.models.word2vec.Vocab object at 0x7f9b5e857ad0>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences)` will run two passes over the sentences iterator. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [7]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences)                       # can be a non-repeatable, 1-pass generator
print(new_model)
print(model.vocab)

2016-11-14 14:56:47,472 : INFO : collecting all words and their counts
2016-11-14 14:56:47,473 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-14 14:56:47,474 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2016-11-14 14:56:47,475 : INFO : Loading a fresh vocabulary
2016-11-14 14:56:47,476 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2016-11-14 14:56:47,477 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2016-11-14 14:56:47,478 : INFO : deleting the raw counts dictionary of 3 items
2016-11-14 14:56:47,479 : INFO : sample=0.001 downsamples 3 most-common words
2016-11-14 14:56:47,480 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2016-11-14 14:56:47,481 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2016-11-14 14:56:47,482 : INFO : resetting layer weights
2016-11-14 14:56:47,483 : INFO : training model with 3 workers o

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.word2vec.Vocab object at 0x7f9b5eb8cd50>, 'first': <gensim.models.word2vec.Vocab object at 0x7f9b2dfb0d50>, 'sentence': <gensim.models.word2vec.Vocab object at 0x7f9b5e857ad0>}


## More data would be nice
For the following examples, we'll use the Lee Corpus (which you already have if you've installed gensim):

In [8]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [9]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

One of them is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [10]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2016-11-14 14:56:52,890 : INFO : collecting all words and their counts
2016-11-14 14:56:52,892 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-14 14:56:52,922 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2016-11-14 14:56:52,925 : INFO : Loading a fresh vocabulary
2016-11-14 14:56:52,941 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2016-11-14 14:56:52,943 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2016-11-14 14:56:52,949 : INFO : deleting the raw counts dictionary of 10186 items
2016-11-14 14:56:52,951 : INFO : sample=0.001 downsamples 54 most-common words
2016-11-14 14:56:52,954 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2016-11-14 14:56:52,955 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2016-11-14 14:56:52,960 : INFO : resetting layer weights
2016-11-14 14:5

In [11]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2016-11-14 14:56:53,391 : INFO : collecting all words and their counts
2016-11-14 14:56:53,393 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-14 14:56:53,431 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2016-11-14 14:56:53,433 : INFO : Loading a fresh vocabulary
2016-11-14 14:56:53,453 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2016-11-14 14:56:53,455 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2016-11-14 14:56:53,465 : INFO : deleting the raw counts dictionary of 10186 items
2016-11-14 14:56:53,467 : INFO : sample=0.001 downsamples 49 most-common words
2016-11-14 14:56:53,470 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2016-11-14 14:56:53,475 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2016-11-14 14:56:53,487 : INFO : resetting layer weights
2016-11-14 14:

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

The last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [12]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2016-11-14 14:56:55,231 : INFO : collecting all words and their counts
2016-11-14 14:56:55,233 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-14 14:56:55,264 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2016-11-14 14:56:55,265 : INFO : Loading a fresh vocabulary
2016-11-14 14:56:55,286 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2016-11-14 14:56:55,287 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2016-11-14 14:56:55,305 : INFO : deleting the raw counts dictionary of 10186 items
2016-11-14 14:56:55,307 : INFO : sample=0.001 downsamples 49 most-common words
2016-11-14 14:56:55,311 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2016-11-14 14:56:55,313 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2016-11-14 14:56:55,326 : INFO : resetting layer weights
2016-11-14 14:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google have released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

Gensim support the same evaluation set, in exactly the same format:

In [13]:
try:
    model.accuracy('./datasets/questions-words.txt')
except:
    raise ValueError("SKIP: Ensure that questions-words.txt is present")

2016-11-14 14:59:04,148 : INFO : precomputing L2-norms of word weight vectors
2016-11-14 14:59:04,163 : INFO : family: 0.0% (0/2)
2016-11-14 14:59:04,224 : INFO : gram3-comparative: 0.0% (0/12)
2016-11-14 14:59:04,251 : INFO : gram4-superlative: 0.0% (0/12)
2016-11-14 14:59:04,283 : INFO : gram5-present-participle: 5.0% (1/20)
2016-11-14 14:59:04,325 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2016-11-14 14:59:04,364 : INFO : gram7-past-tense: 0.0% (0/20)
2016-11-14 14:59:04,396 : INFO : gram8-plural: 0.0% (0/12)
2016-11-14 14:59:04,415 : INFO : total: 1.0% (1/98)


This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.

Once again, **good performance on this test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task.

## Storing and loading models
You can store/load models using the standard gensim methods:

In [14]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2016-11-14 14:59:11,591 : INFO : saving Word2Vec object under /tmp/tmpTikknFgensim_temp, separately None
2016-11-14 14:59:11,592 : INFO : not storing attribute syn0norm
2016-11-14 14:59:11,593 : INFO : not storing attribute cum_table
2016-11-14 14:59:11,632 : INFO : loading Word2Vec object from /tmp/tmpTikknFgensim_temp
2016-11-14 14:59:11,648 : INFO : setting ignored attribute syn0norm to None
2016-11-14 14:59:11,649 : INFO : setting ignored attribute cum_table to None


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:

    model = gensim.models.Word2Vec.load_word2vec_format('/tmp/vectors.txt', binary=False)
    # using gzipped/bz2 input works too, no need to unzip:
    model = gensim.models.Word2Vec.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences:

In [15]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = ['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 
                  'training', 'it', 'with', 'more', 'sentences']
model.train(more_sentences)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2016-11-14 14:59:18,663 : INFO : loading Word2Vec object from /tmp/tmpTikknFgensim_temp
2016-11-14 14:59:18,673 : INFO : setting ignored attribute syn0norm to None
2016-11-14 14:59:18,674 : INFO : setting ignored attribute cum_table to None
2016-11-14 14:59:18,680 : INFO : training model with 4 workers on 1723 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2016-11-14 14:59:18,681 : INFO : expecting 300 sentences, matching count from corpus used for vocabulary survey
2016-11-14 14:59:18,686 : INFO : worker thread finished; awaiting finish of 3 more threads
2016-11-14 14:59:18,687 : INFO : worker thread finished; awaiting finish of 2 more threads
2016-11-14 14:59:18,689 : INFO : worker thread finished; awaiting finish of 1 more threads
2016-11-14 14:59:18,690 : INFO : worker thread finished; awaiting finish of 0 more threads
2016-11-14 14:59:18,691 : INFO : training on 320 raw words (37 effective words) took 0.0s, 5891 effective words/s
2016-11-14 14:59:18,

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [16]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

2016-11-14 14:59:18,791 : INFO : precomputing L2-norms of word weight vectors


[('longer', 0.9873440265655518)]

In [17]:
model.doesnt_match("input is lunch he sentence cat".split())

'sentence'

In [18]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.999209422802
0.995684297997


If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [19]:
model['tree']  # raw NumPy vector of a word

array([ 0.0057368 ,  0.02326648, -0.02958373, -0.00566632, -0.03599387,
       -0.04041369,  0.01113922, -0.0953315 , -0.01736213, -0.02781322,
        0.02952414, -0.02196022, -0.06818835, -0.02112936, -0.01957779,
        0.0493992 , -0.03085088,  0.00034136,  0.02139698, -0.02886172,
       -0.07132264, -0.00490699,  0.07220332,  0.05237336,  0.02018178,
        0.0189541 , -0.00620924,  0.01786778, -0.01942731, -0.00433182,
        0.00136758,  0.07432666,  0.00750727, -0.00523314,  0.02888597,
        0.02185439, -0.05205816,  0.01569716,  0.01176431, -0.06975695,
       -0.0832869 ,  0.01097392, -0.04284792, -0.03217454,  0.03093392,
       -0.05881983,  0.04021532,  0.00108097, -0.01711615,  0.04002371,
        0.0208727 ,  0.00621748, -0.03847566,  0.00535711,  0.01286821,
       -0.02227246,  0.05871677, -0.02759225,  0.01515184, -0.03568283,
       -0.02695257, -0.04913182,  0.02100692, -0.02568992, -0.01113709,
       -0.06927902, -0.07816526,  0.0397783 ,  0.06191112,  0.00

…or en-masse as a 2D NumPy matrix from `model.syn0`.

## Outro
There is a **Bonus App** on the original [blog post](http://rare-technologies.com/word2vec-tutorial/), which runs `word2vec` on the Google News dataset, of **about 100 billion words**.

Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).